In [ ]:
import csv

In [ ]:
dataset = []
with open("access.log") as f:
    reader = csv.reader(f, delimiter=" ")
    for row in reader:
        dataset.append(row)
len(dataset)

In [ ]:
dataset[0]

In [ ]:
dataset_from_ip = {}
for d in dataset:
    dataset_from_ip.setdefault(d[0], [])
    dataset_from_ip[d[0]].append(d)

In [ ]:
len(dataset_from_ip.keys())

In [ ]:
dataset_ip80 = []
for k in dataset_from_ip.keys():
    for d in dataset_from_ip[k][:80]:
        dataset_ip80.append(d)

In [ ]:
len(dataset_ip80)

In [ ]:
import urllib

In [ ]:
def get_vectors(dataset_ip80):
    vectors = []
    for d in dataset_ip80:
        v, req = [], urllib.parse.unquote(d[5], "utf-8")
        # find
        v.append( req.find("%") )
        v.append( req.find(":") )
        # count
        v.append( req.count(":") )
        v.append( req.count("(") )
        v.append( req.count(";") )
        v.append( req.count("%") )
        v.append( req.count("/") )
        v.append( req.count("'") )
        v.append( req.count("<") )
        v.append( req.count("?") )
        v.append( req.count(".") )
        v.append( req.count("#") )
        v.append( req.count("=") )
        v.append( req.count("\\") )
        v.append( req.count(" ") )
        # POST or NOT
        v.append( 1 if req.find("POST") == 0 else 0 )
        # Count of non-alphabetic and non-numeric characters
        f = [ c.isalnum() for c in req ]
        v.append( f.count(False) )
        # Count of non-alphabetic and non-numeric characters in path part
        _pos = req.find(" ")
        v.append( f[_pos+1:].count(False) )
        # Count of non-alphabetic and non-numeric characters in query part
        _pos = req.find("?")
        v.append( f[_pos+1:].count(False) )
        # Length of the most continuous non-alphabetic and non-numeric characters
        _max, _cnt = 0, 0
        for c in f:
            if c == False:
                _cnt += 1
            else:
                if _max < _cnt:
                    _max = _cnt
                _cnt = 0
        if _max < _cnt:
            _max = _cnt
        v.append( _max )
        # count 2bytes string
        v.append( req.count("/%") )
        v.append( req.count("//") )
        v.append( req.count("/.") )
        v.append( req.count("..") )
        v.append( req.count("=/") )
        v.append( req.count("./") )
        v.append( req.count("/?") )
        vectors.append(v)
    return vectors

In [ ]:
vectors = get_vectors(dataset_ip80)
len(vectors)

In [ ]:
import random
import numpy as np
from sklearn.ensemble import IsolationForest

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)

In [ ]:
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import matplotlib.pyplot as plt

In [ ]:
def _cluster_to_graph(X, th=1.0):
    r = linkage(pdist((X)))
    _v = sorted(r[:,2], reverse=True)
    t = th * (sum(_v) / len(_v))
    print(t)
    c = fcluster(r, t, criterion="distance")
    count = {}
    for i, v in enumerate(c):
        count.setdefault(v, 0)
        count[v] += 1
    max_key = max(count.keys())
    max_value = count[max_key]
    del count[max_key]
    for i in range(max_value):
        count.setdefault(max_key, 1)
        max_key += 1
    _x = [ i for i in range(len(count.values())) ]
    _y = sorted(count.values(), reverse=True)
    return _x, _y

In [ ]:
def _plot(_2d, _3d, _th=1.0, sz=600, title="", target=None):
    fig = make_subplots(rows=1, cols=3, 
                        specs=[[{"type": "xy"}, {"type": "scene"}, {}]])
    if type(target) != type(None):
        d, _text = {}, []
        for i, t in enumerate(set(target)):
            d.setdefault(t, i)
        for t in target:
            _text.append(int(d[t]))
        fig.add_trace(
            go.Scatter(x=_2d[:, 0], y=_2d[:, 1], 
                       mode='text', marker=dict(size=5), text=_text), 
            row=1, col=1)
        fig.add_trace(
            go.Scatter3d(x=_3d[:, 0], y=_3d[:, 1], z=_3d[:, 2], 
                         mode='text', marker=dict(size=1), text=_text),
            row=1, col=2)
    else:
        fig.add_trace(
            go.Scatter(x=_2d[:, 0], y=_2d[:, 1], 
                       mode='markers', marker=dict(size=5)), 
            row=1, col=1)
        fig.add_trace(
            go.Scatter3d(x=_3d[:, 0], y=_3d[:, 1], z=_3d[:, 2], 
                         mode='markers', marker=dict(size=1)),
            row=1, col=2)
    _left, _height = _cluster_to_graph(_3d, _th)
    fig.add_trace(go.Bar(x=_left[:100], y=_height[:100], marker={}), row=1, col=3)
    fig.update_layout(height=sz, width=sz*2, title_text=title, showlegend=False)
    fig.show()

# STEP 1

In [ ]:
_x = np.array(vectors)
clf = IsolationForest(random_state=0).fit(_x)
_y = clf.predict(_x)

In [ ]:
_idx = [ i for i in range(len(_x)) ]
random.shuffle(_idx)

In [ ]:
X = [ _x[i] for i in _idx[:10000] ]
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
tsne3d = manifold.TSNE(n_components=3, init='pca', random_state=0)
X_tsne = tsne.fit_transform(X)
X_tsne3d = tsne3d.fit_transform(X)

In [ ]:
_plot(X_tsne, X_tsne3d, 1.0, 400, "Computing t-SNE embedding", 
      [ _y[i] for i in _idx[:10000] ])

In [ ]:
import pickle

In [ ]:
with open("IsolationForest01.pickle", "wb") as f:
    pickle.dump([dataset_ip80, _x, _y], f)

# STEP 2

In [ ]:
with open("IsolationForest01.pickle", "rb") as f:
    dataset_ip80, _x, _y = pickle.load(f)

In [ ]:
len(_x)

In [ ]:
len([ 1 for v in _y if v == -1 ])

In [ ]:
dataset, vectors = [], []
for i in range(len(_y)):
    if _y[i] == -1:
        dataset.append(dataset_ip80[i])
        vectors.append(_x[i])
len(dataset)

In [ ]:
_x = np.array(vectors)
clf = IsolationForest(random_state=0).fit(_x)
_y = clf.predict(_x)

In [ ]:
_idx = [ i for i in range(len(_x)) ]
random.shuffle(_idx)

In [ ]:
X = [ _x[i] for i in _idx[:10000] ]
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
tsne3d = manifold.TSNE(n_components=3, init='pca', random_state=0)
X_tsne = tsne.fit_transform(X)
X_tsne3d = tsne3d.fit_transform(X)

In [ ]:
_plot(X_tsne, X_tsne3d, 1.0, 400, "Computing t-SNE embedding", 
      [ _y[i] for i in _idx[:10000] ])

In [ ]:
with open("IsolationForest02.pickle", "wb") as f:
    pickle.dump([dataset, _x, _y], f)

# STEP 3

In [ ]:
with open("IsolationForest02.pickle", "rb") as f:
    _dataset, _x, _y = pickle.load(f)

In [ ]:
len(_x)

In [ ]:
len([ 1 for v in _y if v == -1 ])

In [ ]:
dataset, vectors = [], []
for i in range(len(_y)):
    if _y[i] == -1:
        dataset.append(_dataset[i])
        vectors.append(_x[i])
len(dataset)

In [ ]:
_x = np.array(vectors)
clf = IsolationForest(random_state=0).fit(_x)
_y = clf.predict(_x)

In [ ]:
_idx = [ i for i in range(len(_x)) ]
random.shuffle(_idx)

In [ ]:
X = [ _x[i] for i in _idx[:10000] ]
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
tsne3d = manifold.TSNE(n_components=3, init='pca', random_state=0)
X_tsne = tsne.fit_transform(X)
X_tsne3d = tsne3d.fit_transform(X)

In [ ]:
_plot(X_tsne, X_tsne3d, 1.0, 400, "Computing t-SNE embedding", 
      [ _y[i] for i in _idx[:10000] ])

# STEP 4

In [ ]:
_x = np.array(vectors)
clf = IsolationForest(random_state=0).fit(_x)
_y = clf.predict(_x)
_yscore = clf.score_samples(_x)

In [ ]:
dataset_from_ip = {}
for i in range(len(dataset)):
    if _y[i] != -1:
        continue
    dataset_from_ip.setdefault(dataset[i][0], [])
    dataset_from_ip[dataset[i][0]].append([_yscore[i], dataset[i]])
print_logs = []
for k in dataset_from_ip.keys():
    d = sorted(dataset_from_ip[k], key=lambda x:x[0])[0]
    print_logs.append(d)
len(print_logs)

In [ ]:
with open("result_by_ip.txt", "w") as f:
    for d in sorted(print_logs, key=lambda x:x[0]):
        f.write(str(d[0]) + " " + " ".join(map(str, d[1])) + "\n")